In [1]:
import pandas as pd
import pickle
import xlrd

## READ IN 1000 ROWS OF FILE (USING SPECIFIC HCPCS CODE) AND CLEAN PROVIDER FILE

In [2]:
%%time

HCPCS_rows =[]
for chunk in pd.read_csv('../data/Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2017.csv', 
                         dtype = {'Zip Code of the Provider': str},
                         chunksize = 1000):
    HCPCS_rows.append(chunk[chunk['HCPCS Code'] == '43249']) 
               
                
hcpcs_pay_43249 = pd.concat(HCPCS_rows, ignore_index=True)

Wall time: 2min 8s


In [3]:
print(hcpcs_pay_43249.shape)
print(hcpcs_pay_43249.info())
hcpcs_pay_43249.head()

(3161, 26)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3161 entries, 0 to 3160
Data columns (total 26 columns):
National Provider Identifier                                3161 non-null int64
Last Name/Organization Name of the Provider                 3161 non-null object
First Name of the Provider                                  2544 non-null object
Middle Initial of the Provider                              1918 non-null object
Credentials of the Provider                                 2504 non-null object
Gender of the Provider                                      2544 non-null object
Entity Type of the Provider                                 3161 non-null object
Street Address 1 of the Provider                            3161 non-null object
Street Address 2 of the Provider                            1675 non-null object
City of the Provider                                        3161 non-null object
Zip Code of the Provider                                    3161 non-null

,National Provider Identifier,Last Name/Organization Name of the Provider,First Name of the Provider,Middle Initial of the Provider,Credentials of the Provider,Gender of the Provider,Entity Type of the Provider,Street Address 1 of the Provider,Street Address 2 of the Provider,City of the Provider,...,HCPCS Code,HCPCS Description,HCPCS Drug Indicator,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount
0,1003011578,RICHARDSON,GAA,O,M.D.,F,I,415 N CENTER ST,SUITE 300,HICKORY,...,43249,Balloon dilation of esophagus using an endoscope,N,26.0,26,26,122.258846,783.000,95.612308,100.033462
1,1003015843,"NORTHBAY PHYSICIAN'S SURGERY CENTER, L.L.C.",NaN,NaN,NaN,NaN,O,1006 NUT TREE ROAD,NaN,VACAVILLE,...,43249,Balloon dilation of esophagus using an endoscope,N,20.0,16,20,814.534500,2408.400,596.400500,447.812500
2,1003040981,SHAH,AKSHAY,V,M.D.,M,I,14155 N 83RD AVE,SUITE 122,PEORIA,...,43249,Balloon dilation of esophagus using an endoscope,N,25.0,24,25,118.805200,479.000,85.632400,84.749600
3,1003041625,GRAHAM,GEOFFREY,G,D.O.,M,I,2216 E 32ND ST,SUITE 201,JOPLIN,...,43249,Balloon dilation of esophagus using an endoscope,N,16.0,15,16,124.320000,728.125,95.510625,101.753125
4,1003078973,TRIVEDI,CHIRAG,D,D.O.,M,I,13640 N 99TH AVE STE 600,SUITE C-3,SUN CITY,...,43249,Balloon dilation of esophagus using an endoscope,N,17.0,16,17,140.833529,632.000,110.417647,112.217647


#### clean up zip code -- added zeroes, keep 1st 5 numbers

In [12]:
hcpcs_pay_43249['zip_fixed'] = hcpcs_pay_43249['Zip Code of the Provider'].str[:5]
hcpcs_pay_43249.head()

,National Provider Identifier,Last Name/Organization Name of the Provider,First Name of the Provider,Middle Initial of the Provider,Credentials of the Provider,Gender of the Provider,Entity Type of the Provider,Street Address 1 of the Provider,Street Address 2 of the Provider,City of the Provider,...,HCPCS Description,HCPCS Drug Indicator,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount,zip_fixed
1,1003015843,"NORTHBAY PHYSICIAN'S SURGERY CENTER, L.L.C.",NaN,NaN,NaN,NaN,O,1006 NUT TREE ROAD,NaN,VACAVILLE,...,Balloon dilation of esophagus using an endoscope,N,20.0,16,20,814.53450,2408.4,596.400500,447.812500,95687
5,1003079187,WASC LLC,NaN,NaN,NaN,NaN,O,3373 COMMERCE PARKWAY,SUITE 1,WOOSTER,...,Balloon dilation of esophagus using an endoscope,N,17.0,15,17,555.31000,1850.0,433.162941,474.892941,44691
6,1003220187,"CENTRAL OHIO ENDOSCOPY CENTER, LLC",NaN,NaN,NaN,NaN,O,3400 OLENTANGY RIVER RD,NaN,COLUMBUS,...,Balloon dilation of esophagus using an endoscope,N,94.0,79,94,596.51000,750.5,463.132234,478.148830,43202
10,1003815242,"DOTHAN SURGERY CENTER, LLC",NaN,NaN,NaN,NaN,O,1450 ROSS CLARK CIR,SUITE 4,DOTHAN,...,Balloon dilation of esophagus using an endoscope,N,446.0,409,446,509.85778,1300.0,391.675538,477.716614,36301
20,1003887225,CENTRAL TEXAS ENDOSCOPY CENTER LLC,NaN,NaN,NaN,NaN,O,2206 E. VILLA MARIA,NaN,BRYAN,...,Balloon dilation of esophagus using an endoscope,N,26.0,23,26,576.83000,1752.0,445.692692,470.542692,77802


#### slice to specific Entity Type=O and Provider Type=Amb Surg Ctr

In [11]:
hcpcs_pay_43249 = hcpcs_pay_43249[hcpcs_pay_43249['Entity Type of the Provider'] == 'O']
print(hcpcs_pay_43249.shape)
hcpcs_pay_43249.head()

(617, 27)


,National Provider Identifier,Last Name/Organization Name of the Provider,First Name of the Provider,Middle Initial of the Provider,Credentials of the Provider,Gender of the Provider,Entity Type of the Provider,Street Address 1 of the Provider,Street Address 2 of the Provider,City of the Provider,...,HCPCS Description,HCPCS Drug Indicator,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount,zip_fixed
1,1003015843,"NORTHBAY PHYSICIAN'S SURGERY CENTER, L.L.C.",NaN,NaN,NaN,NaN,O,1006 NUT TREE ROAD,NaN,VACAVILLE,...,Balloon dilation of esophagus using an endoscope,N,20.0,16,20,814.53450,2408.4,596.400500,447.812500,95687
5,1003079187,WASC LLC,NaN,NaN,NaN,NaN,O,3373 COMMERCE PARKWAY,SUITE 1,WOOSTER,...,Balloon dilation of esophagus using an endoscope,N,17.0,15,17,555.31000,1850.0,433.162941,474.892941,44691
6,1003220187,"CENTRAL OHIO ENDOSCOPY CENTER, LLC",NaN,NaN,NaN,NaN,O,3400 OLENTANGY RIVER RD,NaN,COLUMBUS,...,Balloon dilation of esophagus using an endoscope,N,94.0,79,94,596.51000,750.5,463.132234,478.148830,43202
10,1003815242,"DOTHAN SURGERY CENTER, LLC",NaN,NaN,NaN,NaN,O,1450 ROSS CLARK CIR,SUITE 4,DOTHAN,...,Balloon dilation of esophagus using an endoscope,N,446.0,409,446,509.85778,1300.0,391.675538,477.716614,36301
20,1003887225,CENTRAL TEXAS ENDOSCOPY CENTER LLC,NaN,NaN,NaN,NaN,O,2206 E. VILLA MARIA,NaN,BRYAN,...,Balloon dilation of esophagus using an endoscope,N,26.0,23,26,576.83000,1752.0,445.692692,470.542692,77802


In [25]:
hcpcs_entity_code_asc = hcpcs_pay_43249[hcpcs_pay_43249['Provider Type'] == 'Ambulatory Surgical Center']
print(hcpcs_entity_code_asc.shape)
hcpcs_entity_code_asc.head()

(616, 27)


,National Provider Identifier,Last Name/Organization Name of the Provider,First Name of the Provider,Middle Initial of the Provider,Credentials of the Provider,Gender of the Provider,Entity Type of the Provider,Street Address 1 of the Provider,Street Address 2 of the Provider,City of the Provider,...,HCPCS Description,HCPCS Drug Indicator,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount,zip_fixed
1,1003015843,"NORTHBAY PHYSICIAN'S SURGERY CENTER, L.L.C.",NaN,NaN,NaN,NaN,O,1006 NUT TREE ROAD,NaN,VACAVILLE,...,Balloon dilation of esophagus using an endoscope,N,20.0,16,20,814.53450,2408.4,596.400500,447.812500,95687
5,1003079187,WASC LLC,NaN,NaN,NaN,NaN,O,3373 COMMERCE PARKWAY,SUITE 1,WOOSTER,...,Balloon dilation of esophagus using an endoscope,N,17.0,15,17,555.31000,1850.0,433.162941,474.892941,44691
6,1003220187,"CENTRAL OHIO ENDOSCOPY CENTER, LLC",NaN,NaN,NaN,NaN,O,3400 OLENTANGY RIVER RD,NaN,COLUMBUS,...,Balloon dilation of esophagus using an endoscope,N,94.0,79,94,596.51000,750.5,463.132234,478.148830,43202
10,1003815242,"DOTHAN SURGERY CENTER, LLC",NaN,NaN,NaN,NaN,O,1450 ROSS CLARK CIR,SUITE 4,DOTHAN,...,Balloon dilation of esophagus using an endoscope,N,446.0,409,446,509.85778,1300.0,391.675538,477.716614,36301
20,1003887225,CENTRAL TEXAS ENDOSCOPY CENTER LLC,NaN,NaN,NaN,NaN,O,2206 E. VILLA MARIA,NaN,BRYAN,...,Balloon dilation of esophagus using an endoscope,N,26.0,23,26,576.83000,1752.0,445.692692,470.542692,77802


#### drop unnecessary columns

In [26]:
hcpcs_entity_code_asc = hcpcs_entity_code_asc.drop(columns = ['Middle Initial of the Provider', 
                                                  'Credentials of the Provider',
                                                  'Gender of the Provider',
                                                  'Street Address 2 of the Provider',
                                                  'Zip Code of the Provider'])
print(hcpcs_entity_code_asc.shape)
hcpcs_entity_code_asc.head()

(616, 22)


,National Provider Identifier,Last Name/Organization Name of the Provider,First Name of the Provider,Entity Type of the Provider,Street Address 1 of the Provider,City of the Provider,State Code of the Provider,Country Code of the Provider,Provider Type,Medicare Participation Indicator,...,HCPCS Description,HCPCS Drug Indicator,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount,zip_fixed
1,1003015843,"NORTHBAY PHYSICIAN'S SURGERY CENTER, L.L.C.",NaN,O,1006 NUT TREE ROAD,VACAVILLE,CA,US,Ambulatory Surgical Center,Y,...,Balloon dilation of esophagus using an endoscope,N,20.0,16,20,814.53450,2408.4,596.400500,447.812500,95687
5,1003079187,WASC LLC,NaN,O,3373 COMMERCE PARKWAY,WOOSTER,OH,US,Ambulatory Surgical Center,Y,...,Balloon dilation of esophagus using an endoscope,N,17.0,15,17,555.31000,1850.0,433.162941,474.892941,44691
6,1003220187,"CENTRAL OHIO ENDOSCOPY CENTER, LLC",NaN,O,3400 OLENTANGY RIVER RD,COLUMBUS,OH,US,Ambulatory Surgical Center,Y,...,Balloon dilation of esophagus using an endoscope,N,94.0,79,94,596.51000,750.5,463.132234,478.148830,43202
10,1003815242,"DOTHAN SURGERY CENTER, LLC",NaN,O,1450 ROSS CLARK CIR,DOTHAN,AL,US,Ambulatory Surgical Center,Y,...,Balloon dilation of esophagus using an endoscope,N,446.0,409,446,509.85778,1300.0,391.675538,477.716614,36301
20,1003887225,CENTRAL TEXAS ENDOSCOPY CENTER LLC,NaN,O,2206 E. VILLA MARIA,BRYAN,TX,US,Ambulatory Surgical Center,Y,...,Balloon dilation of esophagus using an endoscope,N,26.0,23,26,576.83000,1752.0,445.692692,470.542692,77802


## READ IN AND CLEAN ZIP COLUMN IN CROSSWALK FILE

In [27]:
zip_crosswalk_df = pd.read_excel('../data/CBSA_ZIP_032020.xlsx', dtype = {'ZIP': str})
zip_crosswalk_df.sample(10)

,CBSA,ZIP,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
45579,99999,79239,0.000017,0.000023,0.000004,1.708045e-05
18780,31860,56082,0.113432,0.104216,0.102995,1.123663e-01
28802,41400,44657,0.017014,0.003926,0.005725,1.577044e-02
16529,29740,88013,0.007221,0.030138,0.000440,8.496549e-03
2282,12580,21020,0.000002,0.000008,0.000000,1.996484e-06
16809,30100,05068,0.012028,0.002159,0.007748,1.061806e-02
34242,47780,47529,0.028283,0.013300,0.012346,2.695663e-02
40386,99999,99693,0.000000,0.000000,0.000004,9.816348e-08
25527,38300,15238,0.006064,0.013092,0.003506,6.486860e-03
32028,44940,29111,0.008718,0.001988,0.000000,8.102228e-03


## READ IN AND CLEAN ASC FILE

In [28]:
hospcharge_apc = pd.read_csv('../data/Provider_Outpatient_Hospital_Charge_Data_by_APC__CY2017.csv', dtype = {'Provider_Zip_Code': str})
print(hospcharge_apc.shape)
hospcharge_apc.head()

(61779, 16)


,Provider_ID,Provider_Name,Provider_Street_Address,Provider_City,Provider_State,Provider_Zip_Code,Provider_HRR,APC,APC_Desc,Beneficiaries,CAPC_Services,Average_Total_Submitted_Charges,Average_Medicare_Allowed_Amount,Average_Medicare_Payment_Amount,Outlier_Services,Average_Medicare_Outlier_Amount
0,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5072,Level 2 Excision/ Biopsy/ Incision and Drainage,249.0,259,9575.01,1038.45,826.28,NaN,NaN
1,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5073,Level 3 Excision/ Biopsy/ Incision and Drainage,52.0,53,12578.28,1792.60,1423.25,NaN,NaN
2,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5091,Level 1 Breast/Lymphatic Surgery and Related P...,26.0,27,11337.61,2113.58,1683.99,0.0,0.0
3,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5092,Level 2 Breast/Lymphatic Surgery and Related P...,23.0,23,17116.16,3737.14,2977.55,0.0,0.0
4,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5112,Level 2 Musculoskeletal Procedures,17.0,17,7382.73,1029.46,820.21,0.0,0.0


#### clean up zip code, add leading zeroes, keep 1st 5 numbers

In [29]:
hospcharge_apc['zip_fixed'] = hospcharge_apc['Provider_Zip_Code'].str[:5]
print(hospcharge_apc.shape)
hospcharge_apc.head(10)

(61779, 17)


,Provider_ID,Provider_Name,Provider_Street_Address,Provider_City,Provider_State,Provider_Zip_Code,Provider_HRR,APC,APC_Desc,Beneficiaries,CAPC_Services,Average_Total_Submitted_Charges,Average_Medicare_Allowed_Amount,Average_Medicare_Payment_Amount,Outlier_Services,Average_Medicare_Outlier_Amount,zip_fixed
0,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5072,Level 2 Excision/ Biopsy/ Incision and Drainage,249.0,259,9575.01,1038.45,826.28,NaN,NaN,36301
1,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5073,Level 3 Excision/ Biopsy/ Incision and Drainage,52.0,53,12578.28,1792.60,1423.25,NaN,NaN,36301
2,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5091,Level 1 Breast/Lymphatic Surgery and Related P...,26.0,27,11337.61,2113.58,1683.99,0.0,0.0,36301
3,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5092,Level 2 Breast/Lymphatic Surgery and Related P...,23.0,23,17116.16,3737.14,2977.55,0.0,0.0,36301
4,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5112,Level 2 Musculoskeletal Procedures,17.0,17,7382.73,1029.46,820.21,0.0,0.0,36301
5,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5113,Level 3 Musculoskeletal Procedures,65.0,68,12500.44,1970.91,1570.32,0.0,0.0,36301
6,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5114,Level 4 Musculoskeletal Procedures,173.0,175,20738.62,4399.54,3502.09,0.0,0.0,36301
7,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5115,Level 5 Musculoskeletal Procedures,59.0,59,32460.51,8078.53,6762.53,0.0,0.0,36301
8,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5153,Level 3 Airway Endoscopy,20.0,20,8477.08,1073.75,855.51,0.0,0.0,36301
9,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5154,Level 4 Airway Endoscopy,28.0,28,9904.53,1974.13,1556.27,0.0,0.0,36301


#### slice to specific APC code=5302

In [30]:
hospcharge_apc = hospcharge_apc[hospcharge_apc['APC'] == 5302]
print(hospcharge_apc.shape)
hospcharge_apc.head()

(2261, 17)


,Provider_ID,Provider_Name,Provider_Street_Address,Provider_City,Provider_State,Provider_Zip_Code,Provider_HRR,APC,APC_Desc,Beneficiaries,CAPC_Services,Average_Total_Submitted_Charges,Average_Medicare_Allowed_Amount,Average_Medicare_Payment_Amount,Outlier_Services,Average_Medicare_Outlier_Amount,zip_fixed
22,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5302,Level 2 Upper GI Procedures,165.0,177,10380.96,1123.67,894.25,0.0,0.0,36301
60,10005,Marshall Medical Center South,2505 U S Highway 431 North,Boaz,AL,35957,AL - Birmingham,5302,Level 2 Upper GI Procedures,48.0,64,3875.07,1148.72,912.36,0.0,0.0,35957
90,10006,Eliza Coffee Memorial Hospital,205 Marengo Street,Florence,AL,35631,AL - Birmingham,5302,Level 2 Upper GI Procedures,193.0,229,5355.04,1048.26,831.00,0.0,0.0,35631
126,10011,St Vincent's East,50 Medical Park East Drive,Birmingham,AL,35235,AL - Birmingham,5302,Level 2 Upper GI Procedures,88.0,89,6510.94,1105.35,868.94,0.0,0.0,35235
141,10012,Dekalb Regional Medical Center,200 Med Center Drive,Fort Payne,AL,35968,AL - Birmingham,5302,Level 2 Upper GI Procedures,78.0,81,9088.11,1143.93,904.27,0.0,0.0,35968


#### John's version of the file -- matches mine (except for fixed zip column)

In [31]:
apc_hospital_5302 = pd.read_csv('../data/apc_hospital_5302.csv')
print(apc_hospital_5302.shape)
apc_hospital_5302.head()

(2261, 16)


,Provider_ID,Provider_Name,Provider_Street_Address,Provider_City,Provider_State,Provider_Zip_Code,Provider_HRR,APC,APC_Desc,Beneficiaries,CAPC_Services,Average_Total_Submitted_Charges,Average_Medicare_Allowed_Amount,Average_Medicare_Payment_Amount,Outlier_Services,Average_Medicare_Outlier_Amount
0,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5302,Level 2 Upper GI Procedures,165.0,177,10380.96,1123.67,894.25,0.0,0.0
1,10005,Marshall Medical Center South,2505 U S Highway 431 North,Boaz,AL,35957,AL - Birmingham,5302,Level 2 Upper GI Procedures,48.0,64,3875.07,1148.72,912.36,0.0,0.0
2,10006,Eliza Coffee Memorial Hospital,205 Marengo Street,Florence,AL,35631,AL - Birmingham,5302,Level 2 Upper GI Procedures,193.0,229,5355.04,1048.26,831.00,0.0,0.0
3,10011,St Vincent's East,50 Medical Park East Drive,Birmingham,AL,35235,AL - Birmingham,5302,Level 2 Upper GI Procedures,88.0,89,6510.94,1105.35,868.94,0.0,0.0
4,10012,Dekalb Regional Medical Center,200 Med Center Drive,Fort Payne,AL,35968,AL - Birmingham,5302,Level 2 Upper GI Procedures,78.0,81,9088.11,1143.93,904.27,0.0,0.0


In [43]:
ascs_cbsa = pd.read_csv('../data/ascs_cbsa.csv')
print(ascs_cbsa.shape)
ascs_cbsa.head()

(682, 32)


,National Provider Identifier,Last Name/Organization Name of the Provider,First Name of the Provider,Middle Initial of the Provider,Credentials of the Provider,Gender of the Provider,Entity Type of the Provider,Street Address 1 of the Provider,Street Address 2 of the Provider,City of the Provider,...,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount,CBSA,ZIP,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,1003015843,"NORTHBAY PHYSICIAN'S SURGERY CENTER, L.L.C.",NaN,NaN,NaN,NaN,O,1006 NUT TREE ROAD,NaN,VACAVILLE,...,814.53450,2408.4,596.400500,447.812500,46700,95687,0.138815,0.073312,0.091418,0.132562
1,1003079187,WASC LLC,NaN,NaN,NaN,NaN,O,3373 COMMERCE PARKWAY,SUITE 1,WOOSTER,...,555.31000,1850.0,433.162941,474.892941,11740,44691,0.000591,0.000000,0.000000,0.000541
2,1003079187,WASC LLC,NaN,NaN,NaN,NaN,O,3373 COMMERCE PARKWAY,SUITE 1,WOOSTER,...,555.31000,1850.0,433.162941,474.892941,49300,44691,0.423806,0.623565,0.705019,0.443312
3,1003220187,"CENTRAL OHIO ENDOSCOPY CENTER, LLC",NaN,NaN,NaN,NaN,O,3400 OLENTANGY RIVER RD,NaN,COLUMBUS,...,596.51000,750.5,463.132234,478.148830,18140,43202,0.011751,0.007771,0.031900,0.012138
4,1003815242,"DOTHAN SURGERY CENTER, LLC",NaN,NaN,NaN,NaN,O,1450 ROSS CLARK CIR,SUITE 4,DOTHAN,...,509.85778,1300.0,391.675538,477.716614,20020,36301,0.235425,0.254917,0.332344,0.239748


In [44]:
ascs_cbsa = ascs_cbsa.drop(columns = ['First Name of the Provider',
                                      'Middle Initial of the Provider', 
                                      'Credentials of the Provider',
                                      'Gender of the Provider',
                                      'Street Address 2 of the Provider',
                                      'Zip Code of the Provider'])
print(ascs_cbsa.shape)
ascs_cbsa.head()

(682, 26)


,National Provider Identifier,Last Name/Organization Name of the Provider,Entity Type of the Provider,Street Address 1 of the Provider,City of the Provider,State Code of the Provider,Country Code of the Provider,Provider Type,Medicare Participation Indicator,Place of Service,...,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount,CBSA,ZIP,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,1003015843,"NORTHBAY PHYSICIAN'S SURGERY CENTER, L.L.C.",O,1006 NUT TREE ROAD,VACAVILLE,CA,US,Ambulatory Surgical Center,Y,F,...,814.53450,2408.4,596.400500,447.812500,46700,95687,0.138815,0.073312,0.091418,0.132562
1,1003079187,WASC LLC,O,3373 COMMERCE PARKWAY,WOOSTER,OH,US,Ambulatory Surgical Center,Y,F,...,555.31000,1850.0,433.162941,474.892941,11740,44691,0.000591,0.000000,0.000000,0.000541
2,1003079187,WASC LLC,O,3373 COMMERCE PARKWAY,WOOSTER,OH,US,Ambulatory Surgical Center,Y,F,...,555.31000,1850.0,433.162941,474.892941,49300,44691,0.423806,0.623565,0.705019,0.443312
3,1003220187,"CENTRAL OHIO ENDOSCOPY CENTER, LLC",O,3400 OLENTANGY RIVER RD,COLUMBUS,OH,US,Ambulatory Surgical Center,Y,F,...,596.51000,750.5,463.132234,478.148830,18140,43202,0.011751,0.007771,0.031900,0.012138
4,1003815242,"DOTHAN SURGERY CENTER, LLC",O,1450 ROSS CLARK CIR,DOTHAN,AL,US,Ambulatory Surgical Center,Y,F,...,509.85778,1300.0,391.675538,477.716614,20020,36301,0.235425,0.254917,0.332344,0.239748


In [ ]:
# look up CBSAs for Nashville
#med_TN = medicare_with_cbsa[medicare_with_cbsa['City of the Provider'] == 'Nashville']

In [46]:
#john did this -- see merged file in data folder
# hosp_5302_with_cbsa = pd.merge(hcpcs_entity_code_asc, zip_crosswalk_df, how='inner', left_on='Provider_Zip_Code')
# hosp_5302_with_cbsa.head()